In [1]:
using LinearAlgebra, Plots, DelimitedFiles
include("hamming.jl")
include("chase.jl")
import .hamming.create_GH
import .chase.SISO_Pyndiah

In [2]:
G, H = create_GH(5)
k, n = size(G)
batch_size = 100
min_num = 30
iter_num = 6
window_size = 7
Α = [0.0, 0.2, 0.3, 0.5, 0.7, 0.9, 1.0, 1.0]

8-element Vector{Float64}:
 0.0
 0.2
 0.3
 0.5
 0.7
 0.9
 1.0
 1.0

In [3]:
function turbo(Y)
    W = zeros(Float64, n, n)
    R = Y + Α[1] * W
    for i = 1:iter_num
        # R = R + Α[i] * W
        if (isodd(i))
            for j = 1:n
                W[j, :] = SISO_Pyndiah(H, R[j, :], i) - Y[j, :]
            end
        end
        if (iseven(i))
            for j = 1:n
                W[:, j] = SISO_Pyndiah(H, R[:, j], i) - Y[:, j]
            end
        end
        R = Y + Α[i+1] * W
    end
    D = ifelse.(R .> 0, 0, 1)
    return D
end

turbo (generic function with 1 method)

In [4]:
function one_iter(σ, algorithm)
    m = rand((0, 1), k, k)
    # if (algorithm == uncoded)
    #     x = m
    # else
    x = mod.(G' * m, 2)
    x = mod.(x * G, 2)
    # end
    y = (-1) .^ x + randn(size(x)) * σ
    z = algorithm(y)
    # println(x)
    # println(z)

    return sum(mod.(x + z, 2))

end
one_iter(1, turbo)

194

In [5]:
function error_rate(σ)
    batch_num = 0
    cnt = 0
    tmp = 0
    while (cnt < min_num)
        tot = 0
        ttt = 0
        A = zeros(Int64, batch_size + 12, Int64(n / 2), Int64(n / 2))
        for i = 2:size(A)[1]
            m = rand((0, 1), Int64(n / 2), k - Int64(n / 2))
            p = mod.([A[i-1, :, :]' m] * G[:, k+1:n], 2)
            A[i, :, :] = [m p]
        end
        B = (-1.0) .^ A
        B = B + randn(size(B)) * σ
        R = copy(B)
        a = zeros(size(B))
        for i = 7:size(B)[1]
            for j = i:-1:i-5
                for l = 1:Int64(n / 2)
                    w = SISO_Pyndiah(H, vec([R[j-1, :, l] R[j, l, :]]), i - j + 1) - vec([B[j-1, :, l] B[j, l, :]])
                    # a[j-1, :, l] = w[1:Int64(n / 2)] * Α[i-j+2]
                    # a[j, l, :] = w[Int64(n / 2)+1:n] * Α[i-j+2]
                    R[j-1, :, l] = B[j-1, :, l] + w[1:Int64(n / 2)] * Α[i-j+2]
                    R[j, l, :] = B[j, l, :] + w[Int64(n / 2)+1:n] * Α[i-j+2]
                end
            end
        end
        D = ifelse.(R .> 0, 0, 1)
        for i = 7:size(A)[1]-6
            if (D[i, :, :] != A[i, :, :])
                tot += 1
                ttt += sum(mod.(D[i, :, :] + A[i, :, :], 2))
            end
        end
        cnt += tot
        tmp += ttt
        batch_num += 1
    end
    p = tmp / (batch_num * batch_size * n / 2 * n / 2)
    return p
end

error_rate (generic function with 1 method)

In [6]:
iter_num = 6
error_rate(10^-0.225)

2.747252747252747e-6